In [1]:
# imports framework
import sys

# imports other libs
import numpy as np
import os
import math
import itertools
import pandas as pd
import sys, os
from evoman.environment import Environment
from demo_controller import player_controller

import random
random.seed(10)
np.random.seed(10)



pygame 2.6.0 (SDL 2.28.4, Python 3.10.14)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def simulation(env,x):
    f,p,e,t = env.play(pcont=x)
    return f

In [3]:
# evaluation
def evaluate(env, x):
    return np.array(list(map(lambda y: simulation(env,y), x)))

In [4]:
def initialize_population(population_size, lower, upper, n_weights = 265):
    return np.random.uniform(lower, upper, (population_size, n_weights))

In [5]:
def parent_selection(pop, pop_fit, n_parents, smoothing = 1):
    fitness  = pop_fit + smoothing - np.min(pop_fit)

    # Fitness proportional selection probability
    fps = fitness / np.sum (fitness)

    # make a random selection of indices
    parent_indices = np.random.choice (np.arange(0,pop.shape[0]), (n_parents,2), p=fps)
    return pop [parent_indices]

In [6]:
def crossover(parents):
    parentsA, parentsB = np.hsplit (parents,2)
    roll = np.random.uniform (size = parentsA.shape)
    offspring = parentsA * (roll >= 0.5) + parentsB * (roll < 0.5)
    # squeeze to get rid of the extra dimension created during parent selecting
    return np.squeeze(offspring,1)

In [7]:
def survivor_selection(pop, pop_fit, n_pop):
    best_fit_indices = np.argsort(pop_fit * -1) # -1 since we are maximizing
    survivor_indices = best_fit_indices [:n_pop]
    return pop [survivor_indices], pop_fit[survivor_indices]

In [8]:
def mutate(pop,min_value,max_value, sigma):
    mutation = np.random.normal(0, sigma, size=pop.shape)
    new_pop = pop + mutation
    new_pop[new_pop>max_value] = max_value
    new_pop[new_pop<min_value] = min_value
    return new_pop

In [22]:
experiment_name = 'test1'

if not os.path.exists(experiment_name):
    os.makedirs(experiment_name)

n_hidden_neurons = 10

# initializes environment for single objective mode (specialist)  with static enemy and ai player
env = Environment(experiment_name=experiment_name,
				  playermode="ai",
				  player_controller=player_controller(n_hidden_neurons),
			  	  speed="fastest",
				  enemymode="static",
				  level=2,
				  visuals=False)


MESSAGE: Pygame initialized for simulation.


In [25]:
population_size = 100
n_evaluations = 3
n_offspring = 50
weight_upper_bound = 2
weight_lower_bound = -2
mutation_sigma = .3
generations = 30

In [26]:
pop = initialize_population(population_size, -2, 2)
pop_fitness = evaluate(env, pop)

for i in range (generations):
    parents=parent_selection(pop, pop_fitness, len(pop_fitness), smoothing = 1)
    offspring = crossover (parents)
    offspring = mutate (offspring, weight_lower_bound, weight_upper_bound, mutation_sigma)
    offspring_fit = evaluate(env, offspring)
    pop = np.vstack((pop,offspring))
    pop_fit = np.concatenate([pop_fitness,offspring_fit])
    pop, pop_fit = survivor_selection(pop, pop_fit, population_size)
    print (f"Gen {i} - Best: {np.max (pop_fit)} - Mean: {np.mean(pop_fit)}")

Gen 0 - Best: 39.013547994715566 - Mean: 0.6466610033798699
Gen 1 - Best: 39.111122041667116 - Mean: -1.023746877545959
Gen 2 - Best: 66.26990021702643 - Mean: -0.6897768843181532
Gen 3 - Best: 39.013547994715566 - Mean: 1.5580356724520659
Gen 4 - Best: 57.19486503108351 - Mean: 0.8157529373104196
Gen 5 - Best: 47.97413402617469 - Mean: 7.01638381406098
Gen 6 - Best: 48.1888590070233 - Mean: 1.3757092583772244
Gen 7 - Best: 48.269900217026425 - Mean: 1.3726349803509816
Gen 8 - Best: 57.01858578874552 - Mean: 2.185870724841304
Gen 9 - Best: 39.17991706964764 - Mean: -0.2399541122145032
Gen 10 - Best: 39.15067522005314 - Mean: -2.2090409002091196
Gen 11 - Best: 39.12226421822036 - Mean: -2.2091604015062773
Gen 12 - Best: 39.1888590070233 - Mean: 0.12965380928831835
Gen 13 - Best: 39.013547994715566 - Mean: -1.6797632727449285
Gen 14 - Best: 39.013547994715566 - Mean: -1.225602751789452
Gen 15 - Best: 39.013547994715566 - Mean: -1.0419543346664966
Gen 16 - Best: 39.026190388130736 - Mean:

In [18]:
fittest_index = np.argmax (pop_fit)
fittest_individual = pop[fittest_index]


In [19]:
#Play with winner

sol = fittest_individual

env = Environment(experiment_name=experiment_name,
				  playermode="ai",
				  player_controller=player_controller(n_hidden_neurons),
			  	  speed="normal",
				  enemymode="static",
				  level=2,
				  visuals=True)

# tests saved demo solutions for each enemy
enemy_list = [1]
for en in enemy_list:
	
	#Update the enemy
	env.update_parameter('enemies',[en])

	env.play(sol)

print('\n  \n')


MESSAGE: Pygame initialized for simulation.

PARAMETER CHANGE: enemies = [1]

  

